# LangGraph : Tools & Integrations

## Complete Guide with Executable Examples

### Topics Covered:
1. **Built-in Tools Overview**
2. **Custom Tool Development**
3. **Provider-Specific Configurations** (Anthropic, OpenAI)
4. **Client-side vs Server-side Tool Execution**
5. **Tool Parameter Handling with extras Attribute**

## Setup and Installation

In [3]:
# Install required packages
!pip install langgraph langchain-anthropic langchain-openai langchain-core -q

In [4]:
# Import required libraries
import os
from typing import Annotated, Literal, TypedDict, Optional, Any
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool, BaseTool, StructuredTool
from pydantic import BaseModel, Field
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
import json
from datetime import datetime

# Set your API keys
# Set your API key
from dotenv import load_dotenv
load_dotenv()

print("✅ Imports completed successfully!")

✅ Imports completed successfully!


---

# 1. Built-in Tools Overview

LangGraph provides several built-in components:

- **ToolNode**: Automatic tool execution
- **tools_condition**: Conditional routing based on tool calls
- **@tool decorator**: Simple function-to-tool conversion
- **StructuredTool**: Complex tool definitions
- **BaseTool**: Base class for custom tools

## 1.1 Define Simple Tools

In [7]:
@tool
def get_current_time() -> str:
    """Get the current time."""
    from datetime import datetime
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"🔧 Tool: get_current_time() → {current_time}")
    return current_time


@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    result = a + b
    print(f"🔧 Tool: add_numbers({a}, {b}) → {result}")
    return result


@tool
def get_random_fact() -> str:
    """Get a random interesting fact."""
    facts = [
        "Honey never spoils. Archaeologists have found 3000-year-old honey that is still edible.",
        "Octopuses have three hearts and blue blood.",
        "Bananas are berries, but strawberries are not.",
        "A group of flamingos is called a flamboyance."
    ]
    import random
    fact = random.choice(facts)
    print(f"🔧 Tool: get_random_fact() → {fact[:50]}...")
    return fact

print("✅ Simple tools defined")

✅ Simple tools defined


## 1.2 Create ToolNode

In [9]:
# Create ToolNode with multiple tools
tools = [get_current_time, add_numbers, get_random_fact]
tool_node = ToolNode(tools)

print(f"✅ ToolNode created with {len(tools)} tools")
print(f"   Tools: {[t.name for t in tools]}")

✅ ToolNode created with 3 tools
   Tools: ['get_current_time', 'add_numbers', 'get_random_fact']


## 1.3 Build Agent with tools_condition

In [11]:
class BasicAgentState(TypedDict):
    messages: Annotated[list, add_messages]


def create_basic_agent_with_tools():
    """Create an agent using built-in tool components"""
    
    llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    def agent_node(state: BasicAgentState):
        print("\n🧠 Agent thinking...")
        response = llm_with_tools.invoke(state["messages"])
        return {"messages": [response]}
    
    # Build graph
    workflow = StateGraph(BasicAgentState)
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", ToolNode(tools))
    
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", tools_condition)
    workflow.add_edge("tools", "agent")
    
    return workflow.compile(checkpointer=MemorySaver())

print("✅ create_basic_agent_with_tools function defined")

✅ create_basic_agent_with_tools function defined


## 1.4 Test the Agent

In [13]:
basic_agent = create_basic_agent_with_tools()

result = basic_agent.invoke(
    {"messages": [HumanMessage(content="What time is it? Also add 42 and 58.")]},
    config={"configurable": {"thread_id": "builtin_1"}}
)

print(f"\n✅ Final Answer: {result['messages'][-1].content}")


🧠 Agent thinking...
🔧 Tool: get_current_time() → 2026-02-08 08:08:07
🔧 Tool: add_numbers(42, 58) → 100

🧠 Agent thinking...

✅ Final Answer: The current time is **2026-02-08 08:08:07**, and 42 + 58 = **100**.


---

# 2. Custom Tool Development

Three ways to create custom tools:

1. **@tool decorator** (simplest)
2. **StructuredTool** (more control)
3. **BaseTool subclass** (maximum flexibility)

## 2.1 Simple Tool with @tool Decorator

In [16]:
@tool
def search_products(query: str, max_results: int = 5) -> str:
    """
    Search for products in the catalog.
    
    Args:
        query: Search query string
        max_results: Maximum number of results to return (default: 5)
    
    Returns:
        JSON string with search results
    """
    products = [
        {"id": 1, "name": "Laptop Pro", "price": 1299, "category": "Electronics"},
        {"id": 2, "name": "Wireless Mouse", "price": 29, "category": "Electronics"},
        {"id": 3, "name": "Desk Chair", "price": 249, "category": "Furniture"},
        {"id": 4, "name": "Monitor 27\"", "price": 399, "category": "Electronics"},
        {"id": 5, "name": "Keyboard RGB", "price": 89, "category": "Electronics"},
    ]
    
    results = [p for p in products if query.lower() in p["name"].lower()]
    results = results[:max_results]
    
    print(f"🔧 Tool: search_products('{query}', max={max_results}) → {len(results)} results")
    return json.dumps(results, indent=2)


# Test the tool
result = search_products.invoke({"query": "Pro", "max_results": 3})
print(result)

🔧 Tool: search_products('Pro', max=3) → 1 results
[
  {
    "id": 1,
    "name": "Laptop Pro",
    "price": 1299,
    "category": "Electronics"
  }
]


## 2.2 StructuredTool with Input Schema

In [18]:
class EmailInput(BaseModel):
    """Input schema for sending emails"""
    to: str = Field(description="Recipient email address")
    subject: str = Field(description="Email subject")
    body: str = Field(description="Email body content")
    priority: Literal["low", "normal", "high"] = Field(
        default="normal",
        description="Email priority level"
    )


def send_email_function(to: str, subject: str, body: str, priority: str = "normal") -> str:
    """Send an email with specified parameters"""
    print(f"📧 Sending email:")
    print(f"   To: {to}")
    print(f"   Subject: {subject}")
    print(f"   Priority: {priority}")
    print(f"   Body length: {len(body)} chars")
    
    return f"Email sent successfully to {to} with priority {priority}"


send_email_tool = StructuredTool.from_function(
    func=send_email_function,
    name="send_email",
    description="Send an email to a specified recipient",
    args_schema=EmailInput,
    return_direct=False
)

# Test the tool
result = send_email_tool.invoke({
    "to": "user@example.com",
    "subject": "Test Email",
    "body": "This is a test message",
    "priority": "high"
})
print(f"✅ Result: {result}")

📧 Sending email:
   To: user@example.com
   Subject: Test Email
   Priority: high
   Body length: 22 chars
✅ Result: Email sent successfully to user@example.com with priority high


## 2.3 Custom BaseTool Subclass

In [20]:
class DatabaseQueryTool(BaseTool):
    """Custom tool for querying a database"""
    
    name: str = "database_query"
    description: str = "Execute a SQL query on the database"
    
    # Custom attributes
    connection_string: str = "mock://database"
    max_results: int = 100
    
    class Config:
        """Pydantic config"""
        arbitrary_types_allowed = True
    
    def _run(self, query: str) -> str:
        """Execute the database query"""
        print(f"🗄️  Database Tool Executing:")
        print(f"   Connection: {self.connection_string}")
        print(f"   Query: {query}")
        print(f"   Max results: {self.max_results}")
        
        if "SELECT" in query.upper():
            results = [
                {"id": 1, "name": "Alice", "role": "Engineer"},
                {"id": 2, "name": "Bob", "role": "Designer"},
            ]
            print(f"   ✅ Found {len(results)} results")
            return json.dumps(results, indent=2)
        else:
            return "Query executed successfully"
    
    async def _arun(self, query: str) -> str:
        """Async version"""
        return self._run(query)


# Create and test
db_tool = DatabaseQueryTool(max_results=50)
result = db_tool.invoke({"query": "SELECT * FROM users WHERE role='Engineer'"})
print(result)

🗄️  Database Tool Executing:
   Connection: mock://database
   Query: SELECT * FROM users WHERE role='Engineer'
   Max results: 50
   ✅ Found 2 results
[
  {
    "id": 1,
    "name": "Alice",
    "role": "Engineer"
  },
  {
    "id": 2,
    "name": "Bob",
    "role": "Designer"
  }
]


C:\Users\LotusBlue\AppData\Local\Temp\ipykernel_28032\2599198561.py:1: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class DatabaseQueryTool(BaseTool):


## 2.4 Tool with Complex Return Types

In [22]:
@tool
def analyze_data(dataset_name: str) -> dict:
    """
    Analyze a dataset and return structured results.
    
    Args:
        dataset_name: Name of the dataset to analyze
    
    Returns:
        Dictionary with analysis results
    """
    print(f"📊 Analyzing dataset: {dataset_name}")
    
    result = {
        "summary": f"Analysis of {dataset_name} completed successfully",
        "key_metrics": {
            "total_records": 1000,
            "average_value": 42.5,
            "std_deviation": 12.3
        },
        "recommendations": [
            "Consider outlier removal",
            "Normalize the data",
            "Check for missing values"
        ],
        "confidence_score": 0.87
    }
    
    print(f"   ✅ Analysis complete (confidence: {result['confidence_score']})")
    return result


# Test
result = analyze_data.invoke({"dataset_name": "sales_q4_2024"})
print(json.dumps(result, indent=2))

📊 Analyzing dataset: sales_q4_2024
   ✅ Analysis complete (confidence: 0.87)
{
  "summary": "Analysis of sales_q4_2024 completed successfully",
  "key_metrics": {
    "total_records": 1000,
    "average_value": 42.5,
    "std_deviation": 12.3
  },
  "recommendations": [
    "Consider outlier removal",
    "Normalize the data",
    "Check for missing values"
  ],
  "confidence_score": 0.87
}


---

# 3. Provider-Specific Configurations

Different LLM providers have different capabilities:

- **Anthropic (Claude)**: Native tool use with strong reasoning
- **OpenAI (GPT)**: Function calling with JSON schema

## 3.1 Anthropic (Claude) Configuration

In [25]:
@tool
def anthropic_example_tool(parameter: str) -> str:
    """Example tool for Anthropic/Claude"""
    return f"Processed with Anthropic: {parameter}"


def create_anthropic_agent():
    """Create agent with Anthropic-specific configuration"""
    
    llm = ChatAnthropic(
        model="claude-sonnet-4-20250514",
        temperature=0,
        max_tokens=1024,
        timeout=30.0,
        max_retries=2,
    )
    
    tools_list = [get_current_time, add_numbers, anthropic_example_tool]
    llm_with_tools = llm.bind_tools(tools_list)
    
    print("✅ Anthropic agent created with:")
    print(f"   Model: claude-sonnet-4-20250514")
    print(f"   Tools: {[t.name for t in tools_list]}")
    print(f"   Tool format: Anthropic native")
    
    return llm_with_tools


anthropic_llm = create_anthropic_agent()

# Test
response = anthropic_llm.invoke([HumanMessage(content="What time is it?")])

if response.tool_calls:
    print(f"✅ Tool calls detected: {[tc['name'] for tc in response.tool_calls]}")
else:
    print(f"✅ Direct response: {response.content[:100]}")

✅ Anthropic agent created with:
   Model: claude-sonnet-4-20250514
   Tools: ['get_current_time', 'add_numbers', 'anthropic_example_tool']
   Tool format: Anthropic native
✅ Tool calls detected: ['get_current_time']


## 3.2 Provider Comparison

In [27]:
provider_comparison = {
    "Anthropic (Claude)": {
        "tool_format": "Native tool use",
        "strengths": [
            "Strong reasoning",
            "Detailed tool use",
            "Good with complex tools"
        ],
        "tool_calling_style": "Explicit tool blocks"
    },
    "OpenAI (GPT)": {
        "tool_format": "Function calling",
        "strengths": [
            "Fast inference",
            "Wide ecosystem",
            "JSON mode support"
        ],
        "tool_calling_style": "Function call objects"
    }
}

for provider, details in provider_comparison.items():
    print(f"\n{provider}:")
    for key, value in details.items():
        if isinstance(value, list):
            print(f"  {key}:")
            for item in value:
                print(f"    - {item}")
        else:
            print(f"  {key}: {value}")


Anthropic (Claude):
  tool_format: Native tool use
  strengths:
    - Strong reasoning
    - Detailed tool use
    - Good with complex tools
  tool_calling_style: Explicit tool blocks

OpenAI (GPT):
  tool_format: Function calling
  strengths:
    - Fast inference
    - Wide ecosystem
    - JSON mode support
  tool_calling_style: Function call objects


---

# 4. Client-Side vs Server-Side Tool Execution

## Two Execution Models:

1. **Client-Side**: Tools run in your application
   - Full control, access to local resources
   - Better security for sensitive data

2. **Server-Side**: Tools run on provider servers (if supported)
   - Lower latency, provider-managed
   - Limited to providers tool set

## 4.1 Client-Side Tool Execution

In [30]:
@tool
def access_local_file(filename: str) -> str:
    """
    Access a local file (can only run client-side).
    
    Args:
        filename: Name of the file to access
    """
    print(f"📁 Client-side: Accessing local file '{filename}'")
    return f"Contents of {filename}: [local data]"


@tool
def query_internal_database(table: str) -> str:
    """
    Query internal database (can only run client-side).
    
    Args:
        table: Database table name
    """
    print(f"🗄️  Client-side: Querying internal database table '{table}'")
    return f"Results from {table}: [sensitive internal data]"


class ClientSideState(TypedDict):
    messages: Annotated[list, add_messages]
    execution_location: str


def create_client_side_agent():
    """Agent with client-side tool execution"""
    
    llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
    client_tools = [access_local_file, query_internal_database]
    llm_with_tools = llm.bind_tools(client_tools)
    
    def agent_node(state: ClientSideState):
        print("\n🧠 Agent (client-side execution)")
        response = llm_with_tools.invoke(state["messages"])
        return {"messages": [response], "execution_location": "client"}
    
    def client_side_tools_node(state: ClientSideState):
        print("\n⚙️  CLIENT-SIDE TOOL EXECUTION")
        print("   Location: Your application server")
        print("   Security: Full control, data stays local")
        
        tool_node = ToolNode(client_tools)
        result = tool_node.invoke(state)
        result["execution_location"] = "client"
        return result
    
    workflow = StateGraph(ClientSideState)
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", client_side_tools_node)
    
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", tools_condition)
    workflow.add_edge("tools", "agent")
    
    return workflow.compile(checkpointer=MemorySaver())


client_agent = create_client_side_agent()

result = client_agent.invoke(
    {"messages": [HumanMessage(content="Access the config.json file")], "execution_location": ""},
    config={"configurable": {"thread_id": "client_1"}}
)

print(f"\n✅ Execution location: {result.get('execution_location', 'unknown')}")


🧠 Agent (client-side execution)

⚙️  CLIENT-SIDE TOOL EXECUTION
   Location: Your application server
   Security: Full control, data stays local
📁 Client-side: Accessing local file 'config.json'

🧠 Agent (client-side execution)

✅ Execution location: client


---

# 5. Tool Parameter Handling with extras Attribute

The **extras** attribute allows passing metadata that:
- Does not appear in the tool schema shown to the LLM
- Can be used for runtime configuration
- Helps with authentication, rate limiting, etc.

## 5.1 Tool with Extras for Configuration

In [33]:
class APIToolWithExtras(BaseTool):
    """Tool that uses extras for configuration"""
    
    name: str = "api_call"
    description: str = "Make an API call with authentication"
    
    # These are extras - not shown to LLM
    api_key: str = "default_key"
    rate_limit: int = 100
    timeout: int = 30
    
    def _run(self, endpoint: str, method: str = "GET") -> str:
        """
        Make API call.
        
        Args:
            endpoint: API endpoint to call
            method: HTTP method
        """
        print(f"🌐 API Call with extras:")
        print(f"   Endpoint: {endpoint}")
        print(f"   Method: {method}")
        print(f"   API Key: {self.api_key[:10]}... (from extras)")
        print(f"   Rate Limit: {self.rate_limit} req/min (from extras)")
        print(f"   Timeout: {self.timeout}s (from extras)")
        
        return f"API response from {endpoint}"


# Create tools with different configurations
production_api = APIToolWithExtras(
    api_key="prod_key_12345",
    rate_limit=1000,
    timeout=60
)

development_api = APIToolWithExtras(
    api_key="dev_key_67890",
    rate_limit=100,
    timeout=30
)

print("\n📞 Production API:")
production_api.invoke({"endpoint": "/users", "method": "GET"})

print("\n📞 Development API:")
development_api.invoke({"endpoint": "/users", "method": "GET"})


📞 Production API:
🌐 API Call with extras:
   Endpoint: /users
   Method: GET
   API Key: prod_key_1... (from extras)
   Rate Limit: 1000 req/min (from extras)
   Timeout: 60s (from extras)

📞 Development API:
🌐 API Call with extras:
   Endpoint: /users
   Method: GET
   API Key: dev_key_67... (from extras)
   Rate Limit: 100 req/min (from extras)
   Timeout: 30s (from extras)


'API response from /users'

## 5.2 Extras for Runtime Context

In [35]:
class ContextAwareTool(BaseTool):
    """Tool that uses extras for runtime context"""
    
    name: str = "user_operation"
    description: str = "Perform an operation with user context"
    
    # Runtime context (not in schema)
    user_id: Optional[str] = None
    session_id: Optional[str] = None
    permissions: list = []
    
    def _run(self, operation: str, data: str) -> str:
        """
        Perform operation with context.
        
        Args:
            operation: Operation to perform
            data: Data for the operation
        """
        print(f"🔐 Context-aware operation:")
        print(f"   Operation: {operation}")
        print(f"   User ID: {self.user_id} (from extras)")
        print(f"   Session: {self.session_id} (from extras)")
        print(f"   Permissions: {self.permissions} (from extras)")
        
        if "write" in self.permissions or "admin" in self.permissions:
            return f"Operation '{operation}' completed for user {self.user_id}"
        else:
            return f"Permission denied for operation '{operation}'"


# Different user contexts
admin_tool = ContextAwareTool(
    user_id="user_123",
    session_id="session_abc",
    permissions=["read", "write", "admin"]
)

readonly_tool = ContextAwareTool(
    user_id="user_456",
    session_id="session_def",
    permissions=["read"]
)

print("\n👤 Admin user operation:")
admin_tool.invoke({"operation": "delete", "data": "record_1"})

print("\n👤 Read-only user operation:")
readonly_tool.invoke({"operation": "delete", "data": "record_1"})


👤 Admin user operation:
🔐 Context-aware operation:
   Operation: delete
   User ID: user_123 (from extras)
   Session: session_abc (from extras)
   Permissions: ['read', 'write', 'admin'] (from extras)

👤 Read-only user operation:
🔐 Context-aware operation:
   Operation: delete
   User ID: user_456 (from extras)
   Session: session_def (from extras)
   Permissions: ['read'] (from extras)


"Permission denied for operation 'delete'"

## 5.3 Multi-Tenant Tool with Extras

In [37]:
class MultiTenantDatabaseTool(BaseTool):
    """Database tool that handles multiple tenants"""
    
    name: str = "database_query"
    description: str = "Query the database"
    
    # Tenant-specific configuration (not in schema)
    tenant_id: str = "default"
    database_name: str = "main_db"
    isolation_level: str = "READ_COMMITTED"
    max_query_time: int = 30
    
    def _run(self, sql_query: str) -> str:
        """
        Execute database query with tenant isolation.
        
        Args:
            sql_query: SQL query to execute
        """
        print(f"🗄️  Multi-tenant database query:")
        print(f"   Tenant: {self.tenant_id}")
        print(f"   Database: {self.database_name}")
        print(f"   Query: {sql_query}")
        
        # Add tenant filtering automatically
        if "WHERE" in sql_query.upper():
            safe_query = sql_query.replace("WHERE", f"WHERE tenant_id='{self.tenant_id}' AND")
        else:
            safe_query = sql_query + f" WHERE tenant_id='{self.tenant_id}'"
        
        print(f"   Safe query: {safe_query}")
        
        return f"Results for tenant {self.tenant_id}: [query results]"


# Create tenant-specific tools
tenant_a_db = MultiTenantDatabaseTool(
    tenant_id="tenant_a",
    database_name="tenant_a_db",
    max_query_time=60
)

tenant_b_db = MultiTenantDatabaseTool(
    tenant_id="tenant_b",
    database_name="tenant_b_db",
    max_query_time=30
)

print("\n🏢 Tenant A query:")
tenant_a_db.invoke({"sql_query": "SELECT * FROM users WHERE active=true"})

print("\n🏢 Tenant B query:")
tenant_b_db.invoke({"sql_query": "SELECT * FROM users WHERE active=true"})


🏢 Tenant A query:
🗄️  Multi-tenant database query:
   Tenant: tenant_a
   Database: tenant_a_db
   Query: SELECT * FROM users WHERE active=true
   Safe query: SELECT * FROM users WHERE tenant_id='tenant_a' AND active=true

🏢 Tenant B query:
🗄️  Multi-tenant database query:
   Tenant: tenant_b
   Database: tenant_b_db
   Query: SELECT * FROM users WHERE active=true
   Safe query: SELECT * FROM users WHERE tenant_id='tenant_b' AND active=true


'Results for tenant tenant_b: [query results]'

---

# Summary

## Key Takeaways:

### 1. Built-in Tools
- Use **ToolNode** for automatic execution
- Use **tools_condition** for routing
- Leverage **@tool** for simple cases

### 2. Custom Tools
- **@tool**: Quick and simple
- **StructuredTool**: Input validation
- **BaseTool**: Maximum control

### 3. Provider Configs
- **Anthropic**: Strong reasoning
- **OpenAI**: Fast inference

### 4. Execution Models
- **Client-side**: Security, control
- **Server-side**: Lower latency

### 5. Extras Attribute
- Configuration not in schema
- Authentication & rate limiting
- Multi-tenant support

## Best Practices:

✅ Clear tool documentation

✅ Use extras for config

✅ Implement error handling

✅ Consider security

✅ Test thoroughly